In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV,train_test_split,KFold
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv("/content/drive/MyDrive/KP_Module/Practical Machine Learning/Practise/Cases/Big_Mart_Sales_Prediction/train.csv")

X_train = data.drop(['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'],axis =1)
y_train = data['Item_Outlet_Sales']

In [3]:
imp_mode = SimpleImputer(strategy='most_frequent')
imp_med = SimpleImputer(strategy='median')
obc = OneHotEncoder(sparse_output=False)


In [4]:
#preproccesing for numerical data
ct_imp  = make_column_transformer(('passthrough',make_column_selector(dtype_include=object)),
                              (imp_med,make_column_selector(dtype_include=['int64','float64'])),
                              verbose_feature_names_out=False).set_output(transform="pandas")

In [5]:
#preprocessing for Categorical data
ct_enc = make_column_transformer((obc,make_column_selector(dtype_include=object)),
                             ("passthrough",make_column_selector(dtype_include=['int64','float64'])),
                             verbose_feature_names_out=False).set_output(transform='pandas')


In [6]:
dtr = DecisionTreeRegressor(random_state=23)
pipe = Pipeline([('Impute',ct_imp),('ENC',ct_enc),('Tree',dtr)])

pipe.fit(X_train,y_train)

Pipeline(steps=[('Impute',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7a7a3fa0f310>),
                                                 ('simpleimputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7a7a3fa0ef50>)],
                                   verbose_feature_names_out=False)),
                ('ENC',
                 ColumnT...former(transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7a7a3fa0f6a0>),
                                                 ('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7a7a3fa0faf0>)],
                                   verbose_feature_names_out=False)),
                ('Tree', DecisionTreeRegressor(random_state=23))])

In [8]:
#Inferencing

test_data = pd.read_csv("/content/drive/MyDrive/KP_Module/Practical Machine Learning/Practise/Cases/Big_Mart_Sales_Prediction/test.csv")
X_test = data.drop(['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'],axis =1)

y_pred = pipe.predict(X_test)

 # create a submission file and check the score

Similar using Bagging

In [9]:
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold,GridSearchCV

X_train1 = ct_enc.fit_transform(X_train)
X_train2 = ct_imp.fit_transform(X_train1)
kfold = KFold(n_splits=5,shuffle=True,random_state=23)
bagg = BaggingRegressor()
print(bagg.get_params())
params = {'estimator':[dtr],
          'n_estimators':[10,15,30,45,50]}
gcv = GridSearchCV(bagg,param_grid=params,cv=kfold,verbose=3,scoring='r2')

gcv.fit(X_train2,y_train)
print("Best Parameters : ",gcv.best_params_)
print("Best Score : ",gcv.best_score_)

{'base_estimator': 'deprecated', 'bootstrap': True, 'bootstrap_features': False, 'estimator': None, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END estimator=DecisionTreeRegressor(random_state=23), n_estimators=10;, score=0.531 total time=   0.5s
[CV 2/5] END estimator=DecisionTreeRegressor(random_state=23), n_estimators=10;, score=0.562 total time=   0.5s
[CV 3/5] END estimator=DecisionTreeRegressor(random_state=23), n_estimators=10;, score=0.500 total time=   0.5s
[CV 4/5] END estimator=DecisionTreeRegressor(random_state=23), n_estimators=10;, score=0.523 total time=   0.5s
[CV 5/5] END estimator=DecisionTreeRegressor(random_state=23), n_estimators=10;, score=0.515 total time=   0.5s
[CV 1/5] END estimator=DecisionTreeRegressor(random_state=23), n_estimators=15;, score=0.544 total time=   0.7s
[CV 2/5] END e